<a href="https://colab.research.google.com/github/awannabecs/PublicTensorflowRepository/blob/main/CHATBOT**.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
### import the will-be-needed modules ####
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [6]:
### load the data ####
data_path =  "/content/drive/MyDrive/chatbotdata/human_text.txt"
data_path2 = "/content/drive/MyDrive/chatbotdata/robot_text.txt"

In [7]:
with open(data_path, mode="r",encoding="utf-8") as f :
  lines = f.read().split("\n")

with open(data_path2, mode="r", encoding="utf-8") as f :
  lines2 = f.read().split("\n")

In [8]:
lines = [re.sub(r"\[\w+\]",'hi',line) for line in lines]
lines = [" ".join(re.findall(r"\w+",line)) for line in lines]

lines2 = [re.sub(r"\[\w+\]",'',line) for line in lines2]
lines2 = [" ".join(re.findall(r"\w+",line)) for line in lines2]

pairs = list(zip(lines,lines2))

random.shuffle(pairs)

In [9]:
input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()

for line in pairs[:400]:
  input_doc, target_doc = line[0],line[1]
  input_docs.append(input_doc)
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)

  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)

input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

In [10]:
input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())

In [11]:
# Maximum length of sentences in input and target documents
max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])


In [12]:
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])


In [13]:
encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float16')


In [14]:
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float16')

In [15]:
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float16')


In [16]:
for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        # Assign 1. for the current line, timestep, & word in encoder_input_data
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.
    
    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.
        if timestep > 0:
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1

In [26]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
#Dimensionality
dimensionality = 256
#The batch size and number of epochs
batch_size = 10
epochs = 100
#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]
#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [27]:
#set the random seed#
import tensorflow as tf
tf.random.set_seed(42)
#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#Compiling
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
#Training
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
training_model.save('/content/drive/MyDrive/chatbotdata/mychatnotmodel_1')

Epoch 1/100
32/32 [==============================] - 34s 924ms/step - loss: 0.4657 - accuracy: 0.0163 - val_loss: 0.4789 - val_accuracy: 0.0092
Epoch 2/100
32/32 [==============================] - 29s 923ms/step - loss: 0.4480 - accuracy: 0.0090 - val_loss: 0.4775 - val_accuracy: 0.0092
Epoch 3/100
32/32 [==============================] - 28s 880ms/step - loss: 0.4445 - accuracy: 0.0090 - val_loss: 0.4767 - val_accuracy: 0.0092
Epoch 4/100
32/32 [==============================] - 28s 890ms/step - loss: 0.4429 - accuracy: 0.0090 - val_loss: 0.4752 - val_accuracy: 0.0092
Epoch 5/100
32/32 [==============================] - 28s 885ms/step - loss: 0.4410 - accuracy: 0.0090 - val_loss: 0.4755 - val_accuracy: 0.0095
Epoch 6/100
32/32 [==============================] - 28s 869ms/step - loss: 0.4396 - accuracy: 0.0090 - val_loss: 0.4755 - val_accuracy: 0.0092
Epoch 7/100
32/32 [==============================] - 28s 867ms/step - loss: 0.4386 - accuracy: 0.0090 - val_loss: 0.4760 - val_accuracy:

INFO:tensorflow:Assets written to: /content/drive/MyDrive/chatbotdata/mychatnotmodel_1/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/chatbotdata/mychatnotmodel_1/assets


In [28]:
model1 = tf.keras.models.load_model('/content/drive/MyDrive/chatbotdata/mychatnotmodel_1')

In [29]:
model1.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None, 1117)  0           []                               
                                ]                                                                 
                                                                                                  
 input_4 (InputLayer)           [(None, None, 1081)  0           []                               
                                ]                                                                 
                                                                                                  
 lstm_2 (LSTM)                  [(None, 256),        1406976     ['input_3[0][0]']                
                                 (None, 256),                                               